## Efficient Branching filter

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 96% !important; }</style>"))

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import numpy.random as npr
import seaborn as sns
%matplotlib inline

In [3]:
params = {
    'mu': -0.04, 
    'nu': 0.01,
    'mean_reversion_coeff': 5.3,
    'rho': -0.7,
    'kappa': -0.5
}

# Stopping time Criterion
epsilon = 1e-5

In [39]:
def branching_particle_filter(S_0, V_0, N, T, r):
    '''
    Theorem-1 Computations
    '''
    mu, nu, mrc, rho, kappa = params['mu'], params['nu'], params['mean_reversion_coeff'], params['rho'], params['kappa']
    '''
    num_particles => N_{t-1}
    num_particles_ => N_{t}
    '''
    ''' Parameters '''
    nu_k = max(int(4*nu/kappa**2 + 0.5), 1)
    a, b, c, d, e = np.sqrt(1 - rho**2), mu - nu*rho/kappa, rho*mrc/kappa - 0.5, rho/kappa, (nu - nu_k)/kappa**2 
    f = e*(kappa**2 - nu - nu_k)/2
    '''Declaration of Variables'''
#     V = np.zeros((N_k)) 
#     logS = np.zeros((N_k)) 
#     logL = np.zeros((N_k)) 
#     Vhat = np.zeros((N_k)) 
#     logShat = np.zeros((N_k)) 
#     logLhat = np.zeros((N_k)) 
    
    V_history = [[] for i in range(T+1)] # V[] will be appended to the history
    logS_history = [[] for i in range(T+1)] # LogS will be appended to the history
    logL_history = [[] for i in range(T+1)] # LogL will be appended to the history
    num_particles = N
    '''Initializing Variables'''
    Vhat = V_0 * np.ones(num_particles) # V[0] = V_0
    logShat = np.log(S_0 * np.ones(num_particles)) 
    logLhat = np.zeros(num_particles) #L[0] = 1
    '''Array containing average weights at each time step'''
    A = np.zeros(T+1) # A[0] is dummy
 
    V = V_history[0] = Vhat
    logS = logS_history[0] = logShat
    logL = logL_history[0] = np.zeros(num_particles)
    
    '''Calculate Vhat, logLhat, logShat terms'''
    Y = np.sqrt(V_0/n) * np.ones((num_particles, n))
    for t in range(1, T+1):
        num_particles_ = 0 # Initializing new particle count 
        Y = np.exp(-mrc/2)*((kappa/2)*npr.randn(num_particles, n)*np.sqrt(delta_t) + Y) # Propagate Y_i^j[t] -> Y_i^j[t+1]
        Vhat = np.sum(Y**2, axis=1)
           
#         print("length of V: ", Vhat.shape)
        # Propagation
        logShat = logS + (a * np.sqrt(V*delta_t)*npr.randn(num_particles) + b * (Vhat - V)*delta_t + d*(Vhat - V)) # Euler Discretization
        logLhat = logL + (e * (np.log(Vhat/V) + mrc)) + (f * (1/Vhat - 1/V) * delta_t) # Simple Euler Discretization
        logLhat = np.log(np.exp(logLhat)/np.sum(np.exp(logLhat)))
#         print("Time: ", t)
        
        A[t] = np.sum(np.exp(logLhat))/N # average weight averaged w.r.t. the "initial # of particles", not num_particles
#         print("Avg: ", A[t])
        logS, V, logL = [], [], [] # Cleaning up the slate to write onto the particle history
        # Select which to branch
        '''
        l -> Number of particles being NOT branched
        '''
        l = 0
        for j in range(0, num_particles): # Non Branched Particles
            if A[t]/r < np.exp(logLhat[j]) < r*A[t]:
                
                if l >= len(logS):
                    logS.append(logShat[j])
                    V.append(Vhat[j])
                    logL.append(logLhat[j]) 
                else:
                    logS[l], V[l], logL[l] = logShat[j], Vhat[j], logLhat[j]
                l += 1
            else: # Send these for branching
                if (j - l) >= len(logS): 
                    logShat = np.append(logShat, logShat[j - l])
                    Vhat = np.append(Vhat, Vhat[j - l])
                    logLhat = np.append(logLhat, logLhat[j - l])
                else: 
                    logShat[j - l], Vhat[j - l], logLhat[j - l] = logShat[j], Vhat[j], logLhat[j]
        # Branching part of the algorithm
        num_particles_ = l # No. of Particles going to be branched
        not_branched = num_particles - l # Number of particles NOT Branched
        if not_branched > 0: 
            
            V_ = npr.uniform(low=0, high=1/not_branched)
            Z = npr.randint(not_branched)

            auxarray1 = np.arange(0, not_branched//2) # Auxilliary array 1 
            auxarray2 = np.arange(not_branched-1, not_branched//2-1, -1) # Auxilliary array 2
            order = 1/not_branched * np.array(alternate_concat(auxarray1 , auxarray2 )) # Set up the ordering
        
        for j in range(0, not_branched):
            U_j = V_ + order[(j + 1 + Z) % not_branched]/not_branched
            N_j = int(np.exp(logLhat[j])/A[t]) + (1 if (U_j <= np.exp(logLhat[j])/A[t] - int(np.exp(logLhat[j])/A[t])) else 0)
#             print("Branches: ", N_j)
            for k in range(0, N_j):
                logS.append(logShat[j]) 
                V.append(Vhat[j]) 
                logL.append(np.log(A[t]))
            num_particles_ += N_j
#         print("Length of V: ", len(V))
        num_particles = num_particles_
        print("Particles:", num_particles)
#         print(t)
        logS_history[t] = logS
        V_history[t] = V
        logL_history[t] = logL
        # Re-setting variables
        V = Vhat = np.array(V)
        logS = logShat = np.array(logS)
        logL = logLhat = np.array(logL)
        Y = np.sqrt(Vhat/n).reshape(num_particles, 1)*np.ones((num_particles, n))

    return (logS_history, V_history, logL_history)

In [41]:
%%time
S_0 = 100
V_0 = 0.01
T = 100
delta_t = 1
n = 200
LogS, V, LogL = branching_particle_filter(S_0, V_0, 100, T, r=1.5)

Particles: 141
Particles: 145
Particles: 149
Particles: 161
Particles: 143
Particles: 114
Particles: 116
Particles: 107
Particles: 158
Particles: 142
Particles: 151
Particles: 142
Particles: 175
Particles: 148
Particles: 131
Particles: 118
Particles: 120
Particles: 53
Particles: 112
Particles: 121
Particles: 110
Particles: 115
Particles: 112
Particles: 106
Particles: 119
Particles: 111
Particles: 150
Particles: 137
Particles: 125
Particles: 116
Particles: 126
Particles: 124
Particles: 147
Particles: 135
Particles: 148
Particles: 141
Particles: 139
Particles: 129
Particles: 135
Particles: 151
Particles: 107
Particles: 149
Particles: 148
Particles: 132
Particles: 127
Particles: 147
Particles: 147
Particles: 144
Particles: 116
Particles: 149
Particles: 143
Particles: 132
Particles: 134
Particles: 156
Particles: 136
Particles: 137
Particles: 138
Particles: 124
Particles: 113
Particles: 146
Particles: 140
Particles: 122
Particles: 123
Particles: 148
Particles: 157
Particles: 145
Particles: 

In [18]:
array1 = np.arange(0, 7//2 + 1)
array2 = np.arange(7, 7//2, -1)

def alternate_concat(array1 , array2):
    temp_array = []
    i, j = 0, 0
    while (i < len(array1)) and (j < len(array2)):
        temp_array.append(array1[i])
        temp_array.append(array2[j])
        i += 1
        j += 1
        
    if i == len(array1):
        while j < len(array2):
            temp_array.append(array2[j])
            j += 1
    else:
        while i < len(array1):
            temp_array.append(array1[i])
            i += 1
    
    return(temp_array)

alternate_concat(array1, array2)

[0, 7, 1, 6, 2, 5, 3, 4]

In [25]:
len(Particles)

NameError: name 'Particles' is not defined

## 